In [ ]:
import os

def read_input(filename):
    with open(filename, "r") as f:
        # dòng đầu tiên
        R, C, F, N, B, T = map(int, f.readline().split())
        rides = []
        for _ in range(N):
            a, b, x, y, s, ff = map(int, f.readline().split())
            rides.append((a, b, x, y, s, ff))
    return R, C, F, N, B, T, rides


def manhattan(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


def solve(R, C, F, N, B, T, rides):
    D = [manhattan((a, b), (x, y)) for (a, b, x, y, s, f) in rides]

    # trạng thái xe
    L = [0] * F           # local time
    X = [0] * F
    Y = [0] * F
    P = [0] * F
    O = [[] for _ in range(F)]

    taken = [False] * N
    score = 0

    while True:
        # chọn xe có local time nhỏ nhất
        t = T
        j = -1
        for idx in range(F):
            if L[idx] < t:
                t = L[idx]
                j = idx

        if t >= T or j == -1:
            break  # tất cả xe xong

        # tìm ride tốt nhất
        m = T
        k = -1
        for i in range(N):
            if taken[i]:
                continue
            a, b, x, y, s, f = rides[i]
            if f >= t + manhattan((X[j], Y[j]), (a, b)) + D[i]:
                M = max(t + manhattan((X[j], Y[j]), (a, b)), s)
                if M < m:
                    m = M
                    k = i

        if k == -1:
            L[j] = T
            continue

        # gán ride k cho xe j
        taken[k] = True
        a, b, x, y, s, f = rides[k]
        O[j].append(k)
        X[j], Y[j] = x, y
        L[j] = m + D[k]
        P[j] += 1
        score += D[k]
        if m == s:
            score += B

    return score, O


def write_output(filename, schedule):
    """Ghi file .out theo format HashCode"""
    with open(filename, "w") as f:
        for rides in schedule:
            f.write(str(len(rides)) + " " + " ".join(map(str, rides)) + "\n")

    # In chú thích ra console
    print("=== Chú thích ===")
    for i, rides in enumerate(schedule):
        print(f"this vehicle {i} is assigned {len(rides)} rides: {rides}")


if __name__ == "__main__":
    # đọc file .in
    filename = "./input/b_should_be_easy.in"
    R, C, F, N, B, T, rides = read_input(filename)

    score, schedule = solve(R, C, F, N, B, T, rides)
    print("Score =", score)
    # print("Schedule:")

    # for j, rs in enumerate(schedule):
    #     print(j, ":", len(rs), "rides:", rs)

    # tạo thư mục output nếu chưa có
    os.makedirs("./output", exist_ok=True)

    # ghi file output vào ./output/
    base = os.path.basename(filename).replace(".in", ".out")
    outname = os.path.join("./output", base)
    write_output(outname, schedule)
    print(f"\nOutput saved to {outname}")


Score = 176877
=== Chú thích ===
this vehicle 0 is assigned 3 rides: [0, 282, 266]
this vehicle 1 is assigned 3 rides: [167, 32, 20]
this vehicle 2 is assigned 3 rides: [296, 214, 196]
this vehicle 3 is assigned 3 rides: [168, 230, 181]
this vehicle 4 is assigned 3 rides: [180, 241, 78]
this vehicle 5 is assigned 3 rides: [216, 102, 242]
this vehicle 6 is assigned 3 rides: [34, 174, 255]
this vehicle 7 is assigned 3 rides: [54, 204, 51]
this vehicle 8 is assigned 3 rides: [108, 29, 238]
this vehicle 9 is assigned 3 rides: [85, 146, 289]
this vehicle 10 is assigned 3 rides: [116, 217, 12]
this vehicle 11 is assigned 3 rides: [275, 128, 265]
this vehicle 12 is assigned 3 rides: [87, 244, 106]
this vehicle 13 is assigned 3 rides: [41, 28, 223]
this vehicle 14 is assigned 3 rides: [184, 172, 31]
this vehicle 15 is assigned 3 rides: [164, 40, 292]
this vehicle 16 is assigned 3 rides: [249, 121, 141]
this vehicle 17 is assigned 3 rides: [99, 284, 250]
this vehicle 18 is assigned 3 rides: [21

In [47]:
import os

def read_input(filename):
    """
    Read the input file and parse the problem parameters.
    Returns: rows, cols, vehicles, number of rides, bonus, total steps, rides list
    """
    with open(filename, "r") as f:
        rows, cols, veh, n_rides, bonus, steps = map(int, f.readline().split())
        rides = []
        for _ in range(n_rides):
            a, b, x, y, s, f_end = map(int, f.readline().split())
            rides.append((a, b, x, y, s, f_end))
    return rows, cols, veh, n_rides, bonus, steps, rides


def manhattan(a, b):
    """
    Compute Manhattan distance between two points a and b.
    """
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


def solve(rows, cols, veh, n_rides, bonus, steps, rides):
    """
    Greedy heuristic solver:
    Assign rides to vehicles based on earliest available vehicle
    and earliest feasible ride start time.
    """
    # Precompute the length of each ride
    ride_len = [manhattan((a, b), (x, y)) for (a, b, x, y, s, f) in rides]

    # Vehicle states
    t_of_the_car = [0] * veh              # current time for each vehicle
    car_x = [0] * veh                     # current row position of each vehicle
    car_y = [0] * veh                     # current column position of each vehicle
    sched = [[] for _ in range(veh)]      # rides assigned to each vehicle

    taken = [False] * n_rides             # ride assignment tracker
    score = 0

    while True:
        # Select the vehicle that becomes free the earliest
        cur_t = steps
        j = -1
        for v in range(veh):
            if t_of_the_car[v] < cur_t:
                cur_t = t_of_the_car[v]
                j = v

        # All vehicles are busy beyond simulation steps
        if cur_t >= steps or j == -1:
            break

        # Find the best feasible ride for this vehicle
        best_start = steps
        k = -1
        for i in range(n_rides):
            if taken[i]:
                continue
            a, b, x, y, s, f = rides[i]

            go = manhattan((car_x[j], car_y[j]), (a, b))  # distance to ride start
            finish = cur_t + go + ride_len[i]             # time to complete this ride

            if finish <= f:  # ride can be finished before its latest finish
                start_time = max(cur_t + go, s)          # wait until earliest start if needed
                if start_time < best_start:             # choose ride that starts earliest
                    best_start = start_time
                    k = i

        # No feasible ride found for this vehicle at this time
        if k == -1:
            t_of_the_car[j] = steps
            continue

        # Assign ride k to vehicle j
        taken[k] = True
        a, b, x, y, s, f = rides[k]

        sched[j].append(k)
        car_x[j], car_y[j] = x, y
        t_of_the_car[j] = best_start + ride_len[k]

        # Update score
        score += ride_len[k]
        if best_start == s:
            score += bonus

    return score, sched


def write_output(filename, sched):
    """
    Write the vehicle schedules to output file.
    """
    with open(filename, "w") as f:
        for rides in sched:
            f.write(str(len(rides)) + " " + " ".join(map(str, rides)) + "\n")

    # Print a summary of assignments
    print("=== Vehicle Ride Assignments ===")
    for i, rides in enumerate(sched):
        print(f"Vehicle {i} is assigned {len(rides)} rides: {rides}")


if __name__ == "__main__":
    filename = "./input/random_instance.in"
    rows, cols, veh, n_rides, bonus, steps, rides = read_input(filename)

    score, sched = solve(rows, cols, veh, n_rides, bonus, steps, rides)
    print("Score =", score)

    os.makedirs("./output", exist_ok=True)
    base = os.path.basename(filename).replace(".in", ".out")
    outname = os.path.join("./output", base)
    write_output(outname, sched)
    print(f"\nOutput saved to {outname}")


Score = 188937
=== Vehicle Ride Assignments ===
Vehicle 0 is assigned 3 rides: [100, 0, 23]
Vehicle 1 is assigned 3 rides: [285, 183, 114]
Vehicle 2 is assigned 4 rides: [35, 17, 157, 110]
Vehicle 3 is assigned 4 rides: [2, 159, 27, 287]
Vehicle 4 is assigned 4 rides: [268, 294, 284, 266]
Vehicle 5 is assigned 3 rides: [144, 197, 47]
Vehicle 6 is assigned 3 rides: [62, 41, 99]
Vehicle 7 is assigned 3 rides: [291, 164, 40]
Vehicle 8 is assigned 3 rides: [153, 237, 84]
Vehicle 9 is assigned 3 rides: [63, 246, 186]
Vehicle 10 is assigned 3 rides: [79, 196, 26]
Vehicle 11 is assigned 3 rides: [129, 13, 55]
Vehicle 12 is assigned 3 rides: [257, 149, 137]
Vehicle 13 is assigned 3 rides: [181, 176, 260]
Vehicle 14 is assigned 3 rides: [80, 68, 147]
Vehicle 15 is assigned 3 rides: [271, 206, 122]
Vehicle 16 is assigned 4 rides: [154, 5, 120, 112]
Vehicle 17 is assigned 4 rides: [94, 204, 67, 258]
Vehicle 18 is assigned 3 rides: [102, 171, 20]
Vehicle 19 is assigned 3 rides: [93, 19, 170]
Vehic

In [46]:
import random

def generate_random_instance(filename, rows=3, cols=4, vehicles=2, n_rides=3,
                                              bonus=2, steps=10, seed=None):
    """
    Generate a random instance and print a human-readable description of rides.
    """
    if seed is not None:
        random.seed(seed)

    rides = []
    for i in range(n_rides):
        a = random.randint(0, rows-1)
        b = random.randint(0, cols-1)
        x = random.randint(0, rows-1)
        y = random.randint(0, cols-1)
        while a == x and b == y:
            x = random.randint(0, rows-1)
            y = random.randint(0, cols-1)

        earliest_start = random.randint(0, steps//2)
        ride_duration = abs(a - x) + abs(b - y)
        latest_finish = earliest_start + ride_duration + random.randint(0, steps//2)
        latest_finish = min(latest_finish, steps)

        rides.append((a, b, x, y, earliest_start, latest_finish))

    # Write to file
    with open(filename, "w") as f:
        f.write(f"{rows} {cols} {vehicles} {n_rides} {bonus} {steps}\n")
        for ride in rides:
            f.write(" ".join(map(str, ride)) + "\n")

    # Print human-readable description
    print(f"{rows} rows, {cols} columns, {vehicles} vehicles, {n_rides} rides, {bonus} bonus and {steps} steps")
    for i, (a, b, x, y, s, f) in enumerate(rides):
        print(f"ride {i+1} from [{a}, {b}] to [{x}, {y}], earliest start {s}, latest finish {f}")

    return rides

def compute_max_possible_score(rides, bonus):
    """
    Compute the maximum possible score assuming:
    - Every ride is completed.
    - Every ride starts at its earliest start (gets bonus).
    """
    max_score = 0
    for a, b, x, y, s, f in rides:
        ride_len = abs(a - x) + abs(b - y)
        max_score += ride_len + bonus
    return max_score


rides = generate_random_instance("./input/random_instance.in", rows=800, cols=1000,
                                 vehicles=100, n_rides=300, bonus=25, steps=25000, seed=42)

max_score = compute_max_possible_score(rides, bonus=25)
print("Maximum possible theoretical score (all rides completed + all bonuses):", max_score)


800 rows, 1000 columns, 100 vehicles, 300 rides, 25 bonus and 25000 steps
ride 1 from [654, 114] to [25, 759], earliest start 4506, latest finish 9792
ride 2 from [228, 142] to [754, 104], earliest start 11087, latest finish 23786
ride 3 from [558, 89] to [604, 432], earliest start 520, latest finish 1397
ride 4 from [95, 223] to [238, 517], earliest start 9863, latest finish 10734
ride 5 from [574, 203] to [733, 665], earliest start 11490, latest finish 21039
ride 6 from [429, 225] to [459, 603], earliest start 4557, latest finish 5071
ride 7 from [777, 825] to [163, 714], earliest start 6924, latest finish 13223
ride 8 from [284, 159] to [220, 980], earliest start 5514, latest finish 8073
ride 9 from [94, 389] to [99, 367], earliest start 5635, latest finish 15553
ride 10 from [270, 826] to [44, 747], earliest start 7527, latest finish 16617
ride 11 from [127, 996] to [387, 80], earliest start 9044, latest finish 15023
ride 12 from [643, 633] to [370, 591], earliest start 3150, lates